In [1]:
from pathlib import Path

import sh
from sh import cp, gunzip, mkdir, rm

from service.config import config

# Будте внимательны этот код используется только в учебных целях

### Код представленный ниже может представлять угрозу безопасности Ваших файлов и устройств. В случае утечки пароля суперпользователя вашего сервера могут наступить необратимые последствия. Он может быть использован третьими лицами в своих целях. Вы можете потерять все данные. Могут возникнуть иные нежелательные последствия. 

In [2]:
# пароль суперпользователя храниться в файле .env в корне проекта и загружается через модуль config

my_password = config['PWD']+ '\n'
my_sudo = sh.sudo.bake("-S", _in=my_password)

## 2. Подготовим список лог фалов авторизации для обработки

In [3]:
log_files = [log_file for log_file in my_sudo.ls('/var/log').split() if 'auth.log' in log_file]
log_files

['auth.log',
 'auth.log.1',
 'auth.log.2.gz',
 'auth.log.3.gz',
 'auth.log.4.gz',
 'auth.log.save',
 'auth.log.save.1']

## 3. Скопируем файлы в дирректорию data_auth

In [4]:
mkdir('data_auth/tmp')

for file_name in log_files:
    my_sudo.cp(f'/var/log/{file_name}', f'data_auth/tmp/{file_name}')
    my_sudo('/bin/chmod', '777', f'data_auth/tmp/{file_name}')

## 4. Разархивируем файлы с расширением gz и обновим список имен

In [5]:
for file_name in log_files:
    if 'gz' not in file_name:
        continue   
    gunzip(f'data_auth/tmp/{file_name}')   

log_files = [file_name.replace('.gz', '') for file_name in log_files]
log_files

['auth.log',
 'auth.log.1',
 'auth.log.2',
 'auth.log.3',
 'auth.log.4',
 'auth.log.save',
 'auth.log.save.1']

## 5. Преобразуем имена всех файлов

In [6]:
for num, file_name in enumerate(log_files):
    if 'save' in file_name:
        print(file_name)
        cp(f'data_auth/tmp/{file_name}', f'data_auth/auth_{num}.log')
        rm(f'data_auth/tmp/{file_name}')
        continue
    words = file_name.split('.')    
    if len(words) < 3:
        new_name = 'auth_0.log'
    else:
        new_name = f'{words[0]}_{words[2]}.{words[1]}'    
    cp(f'data_auth/tmp/{file_name}', f'data_auth/{new_name}')
    rm(f'data_auth/tmp/{file_name}')


auth.log.save
auth.log.save.1


## 6. Удалим временную папку

In [8]:
rm('-R', 'data_auth/tmp')

''

## 7. Посмотрим список полученных файлов 

In [10]:
list(Path('data_auth').glob('*.*'))

[PosixPath('data_auth/auth_1.log'),
 PosixPath('data_auth/auth_5.log'),
 PosixPath('data_auth/auth_0.log'),
 PosixPath('data_auth/auth_2.log'),
 PosixPath('data_auth/auth_3.log'),
 PosixPath('data_auth/auth_4.log'),
 PosixPath('data_auth/auth_6.log')]